# Teste Completo da API de Ações

Este notebook executa uma suíte de testes para todos os endpoints da nossa API, verificando casos de sucesso e de erro.

In [5]:
import requests
import json
import pandas as pd
from IPython.display import display

# --- CONFIGURAÇÕES ---
BASE_URL = "http://127.0.0.1:8000"
# ❗ IMPORTANTE: Coloque aqui uma chave de API válida que você tenha no seu banco de dados
API_KEY = "SUA_CHAVE_DE_API_AQUI" 
HEADERS = {"X-API-Key": API_KEY}

def testar_endpoint(metodo, url, **kwargs):
    """Função auxiliar para testar os endpoints e imprimir os resultados de forma organizada."""
    print(f"--- Testando: {metodo.upper()} {url} ---\n")
    try:
        response = requests.request(metodo, url, **kwargs)
        status_code = response.status_code
        print(f"Status Code: {status_code}")
        
        if response.content:
            try:
                dados = response.json()
                print("Resposta JSON:")
                if isinstance(dados, list) and len(dados) > 0:
                    display(pd.DataFrame(dados))
                else:
                    print(json.dumps(dados, indent=2, ensure_ascii=False))
            except json.JSONDecodeError:
                print("Resposta não é um JSON válido.")
                print(response.text)
        else:
            print("Resposta sem conteúdo (Body).")
            
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
    print("\n" + "="*80 + "\n")

print("Ambiente de teste configurado. Rode as próximas células.")

Ambiente de teste configurado. Rode as próximas células.


## 1. Testes Gerais (Health Check e Listagem)
Verificamos se a API está online e se o endpoint de listagem geral está funcionando.

In [11]:
# Teste de Status da API (não precisa de chave)
testar_endpoint("GET", f"{BASE_URL}/")

# Teste de Listagem de todos os tickers
testar_endpoint("GET", f"{BASE_URL}/api/v1/acoes/listar-todos", headers=HEADERS)

--- Testando: GET http://127.0.0.1:8000/ ---

Status Code: 200
Resposta JSON:
{
  "status": "API online e operacional!"
}


--- Testando: GET http://127.0.0.1:8000/api/v1/acoes/listar-todos ---

Status Code: 200
Resposta JSON:


,codigo,nome
0,ABEV3,Empresa ABEV3
1,B3SA3,Empresa B3SA3
2,BBAS3,Empresa BBAS3
3,BBDC4,Empresa BBDC4
4,EQTL3,Empresa EQTL3
5,GEMI4,Gemini Enterprises SA
6,ITUB4,Empresa ITUB4
7,MGLU3,Empresa MGLU3
8,PETR4,Empresa PETR4
9,RENT3,Empresa RENT3


## 2. Teste de Consulta por Ticker Específico
Testamos a busca por um ticker que existe e por um que não existe para validar o tratamento de erro 404.

In [ ]:
# Teste buscando uma ação que existe
testar_endpoint("GET", f"{BASE_URL}/api/v1/acoes/PETR4", headers=HEADERS)

# Teste buscando uma ação que NÃO existe 
testar_endpoint("GET", f"{BASE_URL}/api/v1/acoes/ACAOINEXISTENTE123", headers=HEADERS)

--- Testando: GET http://127.0.0.1:8000/api/v1/acoes/PETR4 ---

Status Code: 200
Resposta JSON:
{
  "ticker": "PETR4",
  "nome_empresa": "Empresa PETR4",
  "preco_atual": "31.88",
  "variacao_percentual": "0.0",
  "atualizado_em": "2025-06-23T00:00:00"
}


--- Testando: GET http://127.0.0.1:8000/api/v1/acoes/ACAOINEXISTENTE123 ---

Status Code: 404
Resposta JSON:
{
  "detail": "Ticker 'ACAOINEXISTENTE123' ou seu histórico de preços não foram encontrados."
}




## 3. Teste de Consulta de Múltiplos Tickers
Testamos a busca com uma mistura de tickers existentes e não existentes.

url_multi = f"{BASE_URL}/api/v1/acoes/?tickers=PETR4&tickers=VALE3&tickers=NAOEXISTE"
testar_endpoint("GET", url_multi, headers=HEADERS)

## 4. Testes de Dados Relacionados
Verificamos os endpoints de eventos corporativos e de índices.

In [15]:
# Teste de eventos corporativos
testar_endpoint("GET", f"{BASE_URL}/api/v1/acoes/PETR4/eventos", headers=HEADERS)

# Teste dos principais índices
testar_endpoint("GET", f"{BASE_URL}/api/v1/acoes/indices/principais", headers=HEADERS)

--- Testando: GET http://127.0.0.1:8000/api/v1/acoes/PETR4/eventos ---

Status Code: 200
Resposta JSON:
[]


--- Testando: GET http://127.0.0.1:8000/api/v1/acoes/indices/principais ---

Status Code: 404
Resposta JSON:
{
  "detail": "Índices IBOV e IFIX não encontrados no banco de dados."
}




## 5. Teste do Endpoint de Exclusão (DELETE)

**CUIDADO:** A próxima célula irá deletar dados do banco de forma permanente.

In [ ]:
# Ticker coletado será deletado.
TICKER_PARA_DELETAR = "MGLU3" 

print(f"O teste a seguir irá deletar permanentemente a ação '{TICKER_PARA_DELETAR}' e todos os seus dados.")

# Deleta a ação
testar_endpoint("DELETE", f"{BASE_URL}/api/v1/acoes/{TICKER_PARA_DELETAR}", headers=HEADERS)

# Verifica se a ação foi realmente deletada
print(f"Verificando se '{TICKER_PARA_DELETAR}' ainda existe...")
testar_endpoint("GET", f"{BASE_URL}/api/v1/acoes/{TICKER_PARA_DELETAR}", headers=HEADERS)

O teste a seguir irá deletar permanentemente a ação 'MGLU3' e todos os seus dados.
--- Testando: DELETE http://127.0.0.1:8000/api/v1/acoes/MGLU3 ---

Status Code: 200
Resposta JSON:
{
  "detail": "Ação 'MGLU3' e seus dados relacionados foram deletados com sucesso."
}


Verificando se 'MGLU3' ainda existe...
--- Testando: GET http://127.0.0.1:8000/api/v1/acoes/MGLU3 ---

Status Code: 404
Resposta JSON:
{
  "detail": "Ticker 'MGLU3' ou seu histórico de preços não foram encontrados."
}




## 6. Teste do Endpoint de Cadastro de Ticker (POST)

O teste será feito em três partes para garantir que tudo está funcionando como esperado:

1.  **Criação:** Tentamos criar um ticker que ainda não existe. A resposta esperada é `201 Created`.
2.  **Verificação:** Buscamos o ticker recém-criado para confirmar que ele foi salvo no banco. A resposta esperada é `200 OK`.
3.  **Conflito:** Tentamos criar o **mesmo ticker novamente** para garantir que a API retorna o erro `409 Conflict`, como programado.

In [ ]:
NOVO_TICKER = {
    "codigo": "ITSA3",
    "nome": "ITAUSA"
}

# 1. Teste de Criação 
print(f"Tentando criar o ticker '{NOVO_TICKER['codigo']}'...")
testar_endpoint(
    "POST", 
    f"{BASE_URL}/api/v1/acoes/", 
    headers=HEADERS, 
    json=NOVO_TICKER 
)

# 2. Teste de Verificação
print(f"Verificando se o ticker '{NOVO_TICKER['codigo']}' foi criado...")
testar_endpoint(
    "GET", 
    f"{BASE_URL}/api/v1/acoes/{NOVO_TICKER['codigo']}", 
    headers=HEADERS
)

# 3. Teste de Conflito
print(f"Tentando criar o mesmo ticker novamente para testar o erro de conflito...")
testar_endpoint(
    "POST", 
    f"{BASE_URL}/api/v1/acoes/", 
    headers=HEADERS, 
    json=NOVO_TICKER
)

Tentando criar o ticker 'ITSA3'...
--- Testando: POST http://127.0.0.1:8000/api/v1/acoes/ ---

Status Code: 201
Resposta JSON:
{
  "codigo": "ITSA3",
  "nome": "Gemini Enterprises SA"
}


Verificando se o ticker 'ITSA3' foi criado...
--- Testando: GET http://127.0.0.1:8000/api/v1/acoes/ITSA3 ---

Status Code: 404
Resposta JSON:
{
  "detail": "Ticker 'ITSA3' ou seu histórico de preços não foram encontrados."
}


Tentando criar o mesmo ticker novamente para testar o erro de conflito...
--- Testando: POST http://127.0.0.1:8000/api/v1/acoes/ ---

Status Code: 409
Resposta JSON:
{
  "detail": "Ticker já cadastrado."
}


